In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/housesalesprediction/kc_house_data.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This is my first Kaggle project, so I will try to improve it over time 

In [ ]:
data = pd.read_csv('/kaggle/input/housesalesprediction/kc_house_data.csv')

# **Feature Columns**

*     **id** - Unique ID for each home sold
*     **date** - Date of the home sale
*     **price** - Price of each home sold
*     **bedrooms** - Number of bedrooms
*     **bathrooms** - Number of bathrooms, where .5 accounts for a room with a toilet but no shower
*     **sqft_living** - Square footage of the apartments interior living space
*     **sqft_lot**- Square footage of the land space
*     **floors** - Number of floors
*     **waterfront** - A dummy variable for whether the apartment was overlooking the waterfront or not
*     **view** - An index from 0 to 4 of how good the view of the property was
*     **condition** - An index from 1 to 5 on the condition of the apartment,
*     **grade** - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have 
*     a high quality level of construction and design.
*     **sqft_above** - The square footage of the interior housing space that is above ground level
*     **sqft_basement** - The square footage of the interior housing space that is below ground level
*     **yr_built** - The year the house was initially built
*     **yr_renovated** - The year of the house’s last renovation
*     **zipcode** - What zipcode area the house is in
*     **lat** - Lattitude
*     **long** - Longitude
      ** sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
      **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors


# ****Exploratory Data Analysis****

In [ ]:
# first let us see if we have any missing data in each columns
data.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(x= data['bedrooms'])

we can see it is extended u tp 33 but it is a small bar and not visible

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(data['price'])


 as we can see the are some extream points (expensive houses which are not that many) so it is better to skip them when we build the model

In [ ]:
sns.scatterplot(x=data['bedrooms'], y = data['price'])

In [ ]:
# now let us see the correlation of our target with other values 
data.corr()['price'].sort_values()
# we see "sqft_living" has a strong correlation with price 

In [ ]:
sns.scatterplot(x= data['sqft_living'], y =data['price'])

# Geographical Properties

As the house posotion is also effect the price let us look at the lattitude and longitude

In [ ]:
# in our data we have "lattitude" and "longitude" which give the house position in
# king country, USA
plt.figure(figsize=(5,8))
sns.scatterplot(x= data['long'],y =data['lat'], hue = data['price'])

In [ ]:
# because of those extream points we are not getting a good 
# color distribution. so now let us drop those extreame points
data.sort_values('price',ascending=False).head(20)

In [ ]:
len(data)


In [ ]:
non_top_1_perc = data.sort_values('price',ascending=False).iloc[216:]

In [ ]:
# let us plot it, to get more color distribution
plt.figure(figsize=(12,8))
sns.scatterplot(x='long',y='lat',
                data=non_top_1_perc,hue='price',
                palette='RdYlGn',edgecolor=None,alpha=0.2)

In [ ]:
#Other Features
sns.boxplot(x='waterfront',y='price',data=data)

**Working with Feature Data**

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# now let us drop some featuers which is not very informatic
df = data.drop('id',axis=1)

**Feature Engineering from Date**

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
data['month'] = data['date'].apply(lambda date:date.month)

In [ ]:
data['year'] = data['date'].apply(lambda date:date.year)

In [ ]:
sns.boxplot(x='year',y='price',data=data)

In [ ]:
sns.boxplot(x='month',y='price',data=data)

In [ ]:
data.groupby('month').mean()['price'].plot()

In [ ]:
data.groupby('year').mean()['price'].plot()

In [ ]:
data = data.drop('date',axis=1)

In [ ]:
data.columns

In [ ]:

data['zipcode'].value_counts()

In [ ]:
data = data.drop('zipcode',axis=1)

In [ ]:
# could make sense due to scaling, higher should correlate to more value
data['yr_renovated'].value_counts()

In [ ]:
data['sqft_basement'].value_counts()

# **Scaling and Train Test Split**

In [ ]:
X = df.drop('price',axis=1)
y = df['price']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train= scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

# **Creating a Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')